In [18]:
import pandas as pd
from collections import Counter
import glob
import pickle
_ids_dates_genres = pickle.load( open( "pickled_data/ids_dates_genres.p", "rb" ) )
_ids = [p[0] for p in _ids_dates_genres]
df_meta = pd.read_csv("meta/finalmeta.csv")



In [19]:
big_genres = pd.read_csv("meta/datadictionary.csv")
gen_dict = {}
for i in big_genres.itertuples():
    gen_dict[i[1]] = i[2]
gen_dict["chimyst"] = "crime"
gen_dict["locghost"] = "gothic"
gen_dict["lockandkey"] = "crime"

csvs = glob.glob("lavin_results/*.csv")

In [21]:
for datafile in csvs:
    authors = list(df_meta["author"])
    titles = list(df_meta["title"])
    df = pd.read_csv(datafile)
    del df['Unnamed: 0']

    genres_piped = df["genres"]
    genres_main = []
    genres_lavin = []
    for i in genres_piped:
        gen = i.split(" | ")
        g = []
        lavin_gens = []

        for z in gen:
            if "lavin" in z:
                lavin_gens.append(z)

            if z != "teamred" and z!= "teamblack" and z!= "stew" and z != "juvenile" and z != "drop" and "random" not in z:
                #look up and append big genre
                try:
                    g.append(gen_dict[z])
                except: 
                    pass
                    #g.append(z)
        if len(lavin_gens) == 0:
            genres_lavin.append("")
        if len(lavin_gens) == 1:
            genres_lavin.append(lavin_gens[0])
        if len(lavin_gens) > 1:
            genres_lavin.append("lavin_multi")
        #merge duplicates
        g = list(set(g))
        if len(g) > 1:
            final_genre = "multi"
        if len(g) == 0:
            final_genre = "non_genre"
        if len(g) == 1:
            final_genre = g[0]
        genres_main.append((g, final_genre))
    if "horror" in datafile:
        authors_horror = []
        titles_horror = []
        for index in list(df["docid"]):
            pos = index-1
            author = authors[pos]
            title = titles[pos]
            authors_horror.append(author)
            titles_horror.append(title)
        authors = authors_horror
        titles = titles_horror
    print(len(authors), len(genres_main))
    df["author"] = authors
    df["title"] = titles
    df["processed_genre"] = [i[0] for i in genres_main]
    df["final_genre"] = [i[1] for i in genres_main]
    df["lavin_genre"] = [str(i) for i in genres_lavin]
    df_sorted = df.sort_values(by="group_label")
    df_sorted.to_csv(datafile)

984 984
166 166
984 984
166 166
984 984
166 166
984 984
984 984
984 984
984 984
166 166
984 984
166 166
984 984
166 166


In [6]:
df_sorted

,docid,group_label,genres,year,author,title,processed_genre,final_genre,lavin_genre
0,984,0,teamred | chiscifi,1989,"Lesuire, Robert Martin,",The savages of Europe,[scifi],scifi,
337,554,0,teamred | chimyst | cozy,1929,"Green, Anna Katharine,",Agatha Webb,[crime],crime,
336,575,0,chirandom | teamred,1931,"Thornton, Marcellus Eugen","My ""budie"" and I",[],non_genre,
335,576,0,chifantasy | teamblack | chiscifi,1931,"Kipling, Rudyard,",The day's work,[scifi],scifi,
334,577,0,teamblack | chiscifi,1931,"Shiel, M. P.",The yellow danger,[scifi],scifi,
333,578,0,teamred | chiscifi,1931,"James, Henry,",The Turn of the Screw,[scifi],scifi,
332,579,0,chirandom | teamblack | chimyst,1931,"Barr, Amelia Edith Huddle",The lost silver of Briffa,[crime],crime,
331,580,0,teamblack | chiscifi,1931,"Le Braz, Anatole,",Dealings with the dead;,[scifi],scifi,
330,581,0,teamblack | chimyst | stew,1931,"Bangs, John Kendrick,",Ghosts I have met and som,[crime],crime,
329,582,0,chirandom | teamblack,1931,Anonymous1,"Seth Bond, or, A lost tre",[],non_genre,


In [14]:
ratios_all_buckets = []
for bucket in set(list(df_sorted["group_label"])):
    matches = df_sorted.loc[df_sorted['group_label'] == bucket]
    ratios = Counter(list(matches["final_genre"]))
    ratios_all_buckets.append(ratios)

In [15]:
percs = []
for m in ratios_all_buckets:
    p = Counter(dict([(i, m[i] / float(sum(m.values())) * 100.0) for i in m]))
    percs.append(p)

In [16]:
for z in percs:
    #check grou
    print(z.most_common(3))

[('crime', 33.40080971659919), ('non_genre', 33.198380566801625), ('scifi', 28.34008097165992)]
[('crime', 77.77777777777779), ('gothic', 11.11111111111111), ('scifi', 11.11111111111111)]
[('non_genre', 47.16981132075472), ('crime', 30.18867924528302), ('gothic', 13.20754716981132)]
[('non_genre', 100.0)]
[('scifi', 100.0)]
[('scifi', 100.0)]
[('non_genre', 42.857142857142854), ('scifi', 42.857142857142854), ('gothic', 14.285714285714285)]
[('crime', 50.0), ('non_genre', 50.0)]
[('non_genre', 75.0), ('scifi', 25.0)]
[('non_genre', 100.0)]
[('gothic', 50.0), ('crime', 50.0)]
[('scifi', 100.0)]
[('gothic', 66.66666666666666), ('non_genre', 33.33333333333333)]
[('scifi', 100.0)]
[('gothic', 31.25), ('non_genre', 31.25), ('scifi', 18.75)]
[('non_genre', 33.33333333333333), ('scifi', 33.33333333333333), ('crime', 22.22222222222222)]
[('gothic', 80.0), ('non_genre', 15.0), ('multi', 5.0)]
[('gothic', 35.714285714285715), ('scifi', 28.57142857142857), ('non_genre', 21.428571428571427)]
[('non

In [35]:
from scipy.stats.stats import spearmanr
n = [1,8,1,2,3,7,2,5]
v = ["red","blue","red","red","red","blue","red", "blue"]
spearmanr(n,v)

/usr/local/lib/python3.5/site-packages/scipy/stats/stats.py:257: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


SpearmanrResult(correlation=-0.85539892276830165, pvalue=0.0067627766934095042)

In [18]:
from application.selective_features import dictionaries_of_features, make_genres_big
from application.pickles import pickledData
import pandas as pd

In [29]:
import pickle

#define pickle object
class pickledData():
    def __init__(self):
        self.set_data_variables()
    def set_data_variables(self):
        try:
            self._ids_dates_genres = pickle.load( open( "pickled_data/ids_dates_genres.p", "rb" ) )
            self._ids = [p[0] for p in self._ids_dates_genres]
            self.dates = [q[1] for q in self._ids_dates_genres]
            self.genres = [r[2] for r in self._ids_dates_genres]
            # NoteToSelf FOR LATER, add authors to pickle
            #df_meta = pd.read_csv("meta/finalmeta.csv")
            #self.authors = list(df_meta["author"])
        except:
            # get ids, store order here
            self._ids_dates  = [[i.id, i.firstpub] for i in db.session.query(Metadata).all()]
            self.genres = []
            self._ids = [p[0] for p in self._ids_dates]
            self.dates = [q[1] for q in self._ids_dates]
            for _id in self._ids:
                #get genres
                genre_rows  = [i.genre for i in db.session.query(Genres).filter(Genres.work_id==_id).all()]
                #mush genres to string
                g = " | ".join(genre_rows)
                #append
                genre_list.append(g)
            _ids_dates_genres = list(zip(self._ids, self.dates, self.genres))
            pickle.dump( _ids_dates_genres, open( "pickled_data/ids_dates_genres.p", "wb" ) )
        #load feature dict from pickle
        try:
            self.feature_dicts = pickle.load( open( "pickled_data/feature_dicts.p", "rb" ) )
            print("Loaded pickle data successfully.")

        except:
            #\print("Did not find feature data in pickle form. Creating pickle for future use.")
            self.feature_dicts = []

            for _id in self._ids:
                feature_dict = {}
                # get types and counts
                query = "".join(["SELECT type, type_count FROM counts WHERE work_id=", str(_id), " AND type REGEXP '^[A-Za-z]+$';"])
                #loop terms matching certain criteria (regex query)

                conn = pymysql.connect(host='localhost', port=3306, user=USER, passwd=PWD, db='horror')
                cur = conn.cursor()

                a = cur.execute(query)
                for row in cur:
                    #add to dict if ok to use
                    if row[0] in features:
                        feature_dict[row[0]] = row[1]
                self.feature_dicts.append(feature_dict)

            print("Finished making dictionaries")
            pickle.dump( feature_dicts, open( "pickled_data/feature_dicts.p", "wb" ) )
            cur.close()
            conn.close()

def make_genres_big(piped_genres, big_genres):
    import pandas as pd
    
    gen_dict = {}
    for i in big_genres.itertuples():
        gen_dict[i[1]] = i[2]
    gen_dict["chimyst"] = "crime"
    gen_dict["locghost"] = "gothic"
    gen_dict["lockandkey"] = "crime"
    big_genres = []

    for i in piped_genres:
        gen = i.split(" | ")
        g = []
        for z in gen:
            if z != "teamred" and z!= "teamblack" and z!= "stew" and z != "juvenile" and z != "drop" and "random" not in z:
                #look up and append big genre
                try:
                    g.append(gen_dict[z])
                except:
                    pass

        #merge duplicates
        g = list(set(g))
        #assign as multi if still mutiple genres
        if len(g) > 1:
            final_genre = "multi"
        #assign as non_genre if no genres left
        if len(g) == 0:
            final_genre = "non_genre"
        #keep biggenre if length of list is 1
        if len(g) == 1:
            final_genre = g[0]
        big_genres.append(final_genre)
    return big_genres

In [24]:
pData = pickledData()

Loaded pickle data successfully.


In [31]:
big_genre_list = pd.read_csv("meta/datadictionary.csv")
# define as a function, supply a big_genre
def partition_features_test_train(big_genre, pData):
    # Local variables. All are ordered by mysql _id field, which is also the original order of finalmeta.csv
    _ids = pData._ids
    dates = pData.dates
    genres = pData.genres
    #needs conversion
    big_genres = make_genres_big(genres, big_genre_list)

    authors = list(df_meta["author"])
    feature_dicts = pData.feature_dicts

    # partition data into test, train, and feature selection

    # get all ids for big_genre
    for i, j in enumerate(big_genres):
         print(i, _ids[i])
    # "deal out" randomly like deck of cards into three, starting with author repeats
    for_features = []
    test = []
    train = []
partition_features_test_train("crime", pData)

0 1
1 2
2 3
3 4
4 5
5 6
6 7
7 8
8 9
9 10
10 11
11 12
12 13
13 14
14 15
15 16
16 17
17 18
18 19
19 20
20 21
21 22
22 23
23 24
24 25
25 26
26 27
27 28
28 29
29 30
30 31
31 32
32 33
33 34
34 35
35 36
36 37
37 38
38 39
39 40
40 41
41 42
42 43
43 44
44 45
45 46
46 47
47 48
48 49
49 50
50 51
51 52
52 53
53 54
54 55
55 56
56 57
57 58
58 59
59 60
60 61
61 62
62 63
63 64
64 65
65 66
66 67
67 68
68 69
69 70
70 71
71 72
72 73
73 74
74 75
75 76
76 77
77 78
78 79
79 80
80 81
81 82
82 83
83 84
84 85
85 86
86 87
87 88
88 89
89 90
90 91
91 92
92 93
93 94
94 95
95 96
96 97
97 98
98 99
99 100
100 101
101 102
102 103
103 104
104 105
105 106
106 107
107 108
108 109
109 110
110 111
111 112
112 113
113 114
114 115
115 116
116 117
117 118
118 119
119 120
120 121
121 122
122 123
123 124
124 125
125 126
126 127
127 128
128 129
129 130
130 131
131 132
132 133
133 134
134 135
135 136
136 137
137 138
138 139
139 140
140 141
141 142
142 143
143 144
144 145
145 146
146 147
147 148
148 149
149 150
150 151
151 152
15